In [1]:
from convokit import Corpus, download, TextCleaner, TextParser, BoWTransformer
import pandas as pd
import json

In [2]:
corpus = Corpus(filename="/Users/vaughnfranz/.convokit/downloads/supreme-corpus")

## Data Preprocessing 

Below I define a couple of custom cleaning functions to complement the built in cleaning capabilites of convokit.

The TextCleaner transformer from convokit operates on a string, so these functions do as well. 

The first function removes punctuation. The second removes stopwords and lemmatizes the text.

In [3]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
stop_words = set(stopwords.words('english'))
wordNetLemm = WordNetLemmatizer()

def remove_punctuation(text):
    cleaned = "".join([char for char in text if char not in string.punctuation])
    return cleaned

def custom_cleaner(text):
    toks = text.split()
    toks = [word for word in toks if not word in stop_words]
    toks = [wordNetLemm.lemmatize(word) for word in toks]
    cleaned = " ".join(toks)
    return cleaned

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vaughnfranz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/vaughnfranz/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/vaughnfranz/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


I'm going to start wiht removing the punctuation, then use the built in cleaning functionality, and then do the additional cleaning. The reason for this is to avoid affecting the special tokens which convokit inserts for numbers and the like.

Verbosity of 250,000 feels reasonable for 1.7+ million utterances, so I will use that number throughout. 

The TextCleaner class of convokit takes a keyword arg that allows us to specify a custom cleaining function to apply to each utterance. 

With replace set to False the cleaner should store the cleaned text in an attribute on the utterances called 'cleaned.' The original text will be preserved in 'text.' I will then do the additional cleaning steps on the 'cleaned' attribute. 

In [4]:
corpus = TextCleaner(verbosity=250000, text_cleaner=remove_punctuation, replace_text=False).transform(corpus)

250000/1700789 utterances processed
500000/1700789 utterances processed
750000/1700789 utterances processed
1000000/1700789 utterances processed
1250000/1700789 utterances processed
1500000/1700789 utterances processed
1700789/1700789 utterances processed


Let's make sure the cleaning is operating more or less as expected.

In [5]:
test_utterance_id = '13127__0_000'
utt = corpus.get_utterance(test_utterance_id)
print('ORIGINAL:', utt.text)
print('CLEANED:', utt.meta['cleaned'])

ORIGINAL: Number 71, Lonnie Affronti versus United States of America.
Mr. Murphy.
CLEANED: Number 71 Lonnie Affronti versus United States of America
Mr Murphy


Cleaning using the built in functionality of convokit. 

The TextCleaner will, by default:
- fix unicode errors, transliterate text to the closest ASCII representation
- lowercase text
- remove line breaks
- replace URLs, emails, phone numbers, numbers, and currency symbols with special tokens

The cleaner will operate by default on the utterances (specifically, utterance.text).

In [6]:
corpus = TextCleaner(verbosity=250000, input_field='cleaned', replace_text=False).transform(corpus)

250000/1700789 utterances processed
500000/1700789 utterances processed
750000/1700789 utterances processed
1000000/1700789 utterances processed
1250000/1700789 utterances processed
1500000/1700789 utterances processed
1700789/1700789 utterances processed


Another sanity test on the cleaning process.

In [7]:
test_utterance_id = '13127__0_000'
test_utterance_id_2 = '13127__0_004'
utt = corpus.get_utterance(test_utterance_id)
print('TEST: 13127__0_000')
print('ORIGINAL:', utt.text)
print('CLEANED:', utt.meta['cleaned'])
utt2 = corpus.get_utterance(test_utterance_id_2)
print('TEST: 13127__0_004')
print('ORIGINAL:', utt2.text)
print('CLEANED:', utt2.meta['cleaned'])

TEST: 13127__0_000
ORIGINAL: Number 71, Lonnie Affronti versus United States of America.
Mr. Murphy.
CLEANED: number <number> lonnie affronti versus united states of america mr murphy
TEST: 13127__0_004
ORIGINAL: Was the aggregate prison sentence was 20 or 25 years?
CLEANED: was the aggregate prison sentence was <number> or <number> years


Now performing our other custom cleaning steps as defined in the function up top. 

In [8]:
corpus = TextCleaner(verbosity=250000, text_cleaner=custom_cleaner, input_field='cleaned', replace_text=False).transform(corpus)

250000/1700789 utterances processed
500000/1700789 utterances processed
750000/1700789 utterances processed
1000000/1700789 utterances processed
1250000/1700789 utterances processed
1500000/1700789 utterances processed
1700789/1700789 utterances processed


Sanity test again...

In [9]:
test_utterance_id = '13127__0_000'
test_utterance_id_2 = '13127__0_004'
utt = corpus.get_utterance(test_utterance_id)
print('TEST: 13127__0_000')
print('ORIGINAL:', utt.text)
print('CLEANED:', utt.meta['cleaned'])
utt2 = corpus.get_utterance(test_utterance_id_2)
print('TEST: 13127__0_004')
print('ORIGINAL:', utt2.text)
print('CLEANED:', utt2.meta['cleaned'])

TEST: 13127__0_000
ORIGINAL: Number 71, Lonnie Affronti versus United States of America.
Mr. Murphy.
CLEANED: number <number> lonnie affronti versus united state america mr murphy
TEST: 13127__0_004
ORIGINAL: Was the aggregate prison sentence was 20 or 25 years?
CLEANED: aggregate prison sentence <number> <number> year


### OMIT THIS -- DOES NOT WORK AS ANTICIPATED 

Now, we use the TextParser to tokenize our cleaned strings. 

I'm going to use a custom tokenizer here which simply splits the sentences on spaces. This will give us our desired output as a list of strings which is needed for the gensim Word2Vec model. 

This uses nltk's sentence tokenizer by default. The output will be stored in a field called 'parsed.'

If you want to run this you need to make sure spacy's english model is downloaded. You can do this by running:

``` python -m spacy download en ```

In [13]:
def custom_tokenizer(text):
    toks = text.split()
    return toks 
# corpus = TextParser(verbosity=250000, sent_tokenizer=custom_tokenizer, input_field='cleaned', mode='tokenize').transform(corpus)

## Putting the data together 
Now we can get our dataframes, and connect the case information with the utterances.

In [15]:
utterances_df = corpus.get_utterances_dataframe()

Dropping the parsed column from the misstep in using convokit's TextParser earlier... You don't need to run this if you skipped the earlier step. 

In [26]:
utterances_df = utterances_df.drop(columns=['meta.parsed'])

Saving the dataframe to a file for safe keeping so that the above steps do not need to be repeated. 

In [18]:
utterances_df.to_csv(path_or_buf="parsed_utterances.csv", index=False)

Now lets actually tokenize our text, using the custom tokenizer defined earlier. 

In [19]:
utterances_df['tokens'] = utterances_df['meta.cleaned'].apply(custom_tokenizer)

In [27]:
utterances_df.head()

,timestamp,text,speaker,reply_to,conversation_id,meta.case_id,meta.start_times,meta.stop_times,meta.speaker_type,meta.side,meta.timestamp,meta.cleaned,vectors,tokens,meta.win_side,meta.votes_side
0,None,"Number 71, Lonnie Affronti versus United State...",j__earl_warren,None,13127,1955_71,"[0.0, 7.624]","[7.624, 9.218]",J,None,0.0,number <number> lonnie affronti versus united ...,[],"[number, <number>, lonnie, affronti, versus, u...",0,"{'j__john_m_harlan2': 0, 'j__hugo_l_black': 0,..."
1,None,May it please the Court.\nWe are here by writ ...,harry_f_murphy,13127__0_000,13127,1955_71,"[9.218, 11.538, 15.653, 22.722, 28.849, 33.575]","[11.538, 15.653, 22.722, 28.849, 33.575, 48.138]",A,1,9.218,may please court writ certiorari eighth circui...,[],"[may, please, court, writ, certiorari, eighth,...",0,"{'j__john_m_harlan2': 0, 'j__hugo_l_black': 0,..."
2,None,Consecutive sentences.,j__william_o_douglas,13127__0_001,13127,1955_71,[48.138],[49.315],J,None,48.138,consecutive sentence,[],"[consecutive, sentence]",0,"{'j__john_m_harlan2': 0, 'j__hugo_l_black': 0,..."
3,None,"Consecutive sentences.\nIn this case, the defe...",harry_f_murphy,13127__0_002,13127,1955_71,"[49.315, 51.844, 60.81, 67.083, 72.584, 89.839...","[51.844, 60.81, 67.083, 72.584, 89.839, 95.873...",A,1,49.315,consecutive sentence case defendant affronti i...,[],"[consecutive, sentence, case, defendant, affro...",0,"{'j__john_m_harlan2': 0, 'j__hugo_l_black': 0,..."
4,None,Was the aggregate prison sentence was 20 or 25...,<INAUDIBLE>,13127__0_003,13127,1955_71,[174.058],[176.766],None,None,174.058,aggregate prison sentence <number> <number> year,[],"[aggregate, prison, sentence, <number>, <numbe...",0,"{'j__john_m_harlan2': 0, 'j__hugo_l_black': 0,..."


In [21]:
conversations_df = corpus.get_conversations_dataframe()

In [22]:
conversations_df.head()

,vectors,meta.case_id,meta.advocates,meta.win_side,meta.votes_side
id,,,,,
13127,[],1955_71,"{'harry_f_murphy': {'side': 1, 'role': 'inferr...",0,"{'j__john_m_harlan2': 0, 'j__hugo_l_black': 0,..."
12997,[],1955_410,"{'howard_c_westwood': {'side': 1, 'role': 'inf...",1,"{'j__john_m_harlan2': 1, 'j__hugo_l_black': 1,..."
13024,[],1955_410,"{'howard_c_westwood': {'side': 1, 'role': 'inf...",1,"{'j__john_m_harlan2': 1, 'j__hugo_l_black': 1,..."
13015,[],1955_351,"{'harry_d_graham': {'side': 3, 'role': 'inferr...",1,"{'j__john_m_harlan2': 1, 'j__hugo_l_black': 1,..."
13016,[],1955_38,"{'robert_n_gorman': {'side': 3, 'role': 'infer...",0,"{'j__john_m_harlan2': 0, 'j__hugo_l_black': 0,..."


We can use the pandas built in merge function to bring in the case information to the utterances df. 

In [23]:
utterances_df = pd.merge(utterances_df, conversations_df[['meta.case_id', 'meta.win_side', 'meta.votes_side']], how='left', left_on='meta.case_id', right_on='meta.case_id')

In [28]:
utterances_df.head()

,timestamp,text,speaker,reply_to,conversation_id,meta.case_id,meta.start_times,meta.stop_times,meta.speaker_type,meta.side,meta.timestamp,meta.cleaned,vectors,tokens,meta.win_side,meta.votes_side
0,None,"Number 71, Lonnie Affronti versus United State...",j__earl_warren,None,13127,1955_71,"[0.0, 7.624]","[7.624, 9.218]",J,None,0.0,number <number> lonnie affronti versus united ...,[],"[number, <number>, lonnie, affronti, versus, u...",0,"{'j__john_m_harlan2': 0, 'j__hugo_l_black': 0,..."
1,None,May it please the Court.\nWe are here by writ ...,harry_f_murphy,13127__0_000,13127,1955_71,"[9.218, 11.538, 15.653, 22.722, 28.849, 33.575]","[11.538, 15.653, 22.722, 28.849, 33.575, 48.138]",A,1,9.218,may please court writ certiorari eighth circui...,[],"[may, please, court, writ, certiorari, eighth,...",0,"{'j__john_m_harlan2': 0, 'j__hugo_l_black': 0,..."
2,None,Consecutive sentences.,j__william_o_douglas,13127__0_001,13127,1955_71,[48.138],[49.315],J,None,48.138,consecutive sentence,[],"[consecutive, sentence]",0,"{'j__john_m_harlan2': 0, 'j__hugo_l_black': 0,..."
3,None,"Consecutive sentences.\nIn this case, the defe...",harry_f_murphy,13127__0_002,13127,1955_71,"[49.315, 51.844, 60.81, 67.083, 72.584, 89.839...","[51.844, 60.81, 67.083, 72.584, 89.839, 95.873...",A,1,49.315,consecutive sentence case defendant affronti i...,[],"[consecutive, sentence, case, defendant, affro...",0,"{'j__john_m_harlan2': 0, 'j__hugo_l_black': 0,..."
4,None,Was the aggregate prison sentence was 20 or 25...,<INAUDIBLE>,13127__0_003,13127,1955_71,[174.058],[176.766],None,None,174.058,aggregate prison sentence <number> <number> year,[],"[aggregate, prison, sentence, <number>, <numbe...",0,"{'j__john_m_harlan2': 0, 'j__hugo_l_black': 0,..."


In [29]:
utterances_df['meta.win_side'].value_counts()

 1    1378597
 0     796114
 2        831
-1        374
Name: meta.win_side, dtype: int64

According to convokit documentation a 2 signifies that the decision was unclear and a -1 signifies that the data was unavailable. We can drop these rows to simplify the classification task. 

In [30]:
utterances_df = utterances_df[utterances_df['meta.win_side'] != 2]
utterances_df = utterances_df[utterances_df['meta.win_side'] != -1]

In [31]:
utterances_df['meta.win_side'].value_counts()

1    1378597
0     796114
Name: meta.win_side, dtype: int64

We are also going to make use of the speaker information in our models, so let's examine that and drop data where necessary.

In [32]:
utterances_df['meta.speaker_type'].value_counts()

A    1056973
J    1012327
Name: meta.speaker_type, dtype: int64

We also need to do a one-hot-encoding of the speakers for our model. Doing that here.

In [33]:
utterances_df = pd.get_dummies(utterances_df, prefix=['speaker_type'], columns=['meta.speaker_type'])

In [34]:
utterances_df.head()

,timestamp,text,speaker,reply_to,conversation_id,meta.case_id,meta.start_times,meta.stop_times,meta.side,meta.timestamp,meta.cleaned,vectors,tokens,meta.win_side,meta.votes_side,speaker_type_A,speaker_type_J
0,None,"Number 71, Lonnie Affronti versus United State...",j__earl_warren,None,13127,1955_71,"[0.0, 7.624]","[7.624, 9.218]",None,0.0,number <number> lonnie affronti versus united ...,[],"[number, <number>, lonnie, affronti, versus, u...",0,"{'j__john_m_harlan2': 0, 'j__hugo_l_black': 0,...",0,1
1,None,May it please the Court.\nWe are here by writ ...,harry_f_murphy,13127__0_000,13127,1955_71,"[9.218, 11.538, 15.653, 22.722, 28.849, 33.575]","[11.538, 15.653, 22.722, 28.849, 33.575, 48.138]",1,9.218,may please court writ certiorari eighth circui...,[],"[may, please, court, writ, certiorari, eighth,...",0,"{'j__john_m_harlan2': 0, 'j__hugo_l_black': 0,...",1,0
2,None,Consecutive sentences.,j__william_o_douglas,13127__0_001,13127,1955_71,[48.138],[49.315],None,48.138,consecutive sentence,[],"[consecutive, sentence]",0,"{'j__john_m_harlan2': 0, 'j__hugo_l_black': 0,...",0,1
3,None,"Consecutive sentences.\nIn this case, the defe...",harry_f_murphy,13127__0_002,13127,1955_71,"[49.315, 51.844, 60.81, 67.083, 72.584, 89.839...","[51.844, 60.81, 67.083, 72.584, 89.839, 95.873...",1,49.315,consecutive sentence case defendant affronti i...,[],"[consecutive, sentence, case, defendant, affro...",0,"{'j__john_m_harlan2': 0, 'j__hugo_l_black': 0,...",1,0
4,None,Was the aggregate prison sentence was 20 or 25...,<INAUDIBLE>,13127__0_003,13127,1955_71,[174.058],[176.766],None,174.058,aggregate prison sentence <number> <number> year,[],"[aggregate, prison, sentence, <number>, <numbe...",0,"{'j__john_m_harlan2': 0, 'j__hugo_l_black': 0,...",0,0


In [47]:
utterances_df['meta.case_id'].value_counts()

1961_8          12672
1961_2           7476
1965_759         6595
1960_6           5576
1962_8           4972
                ...  
1977_76-5935       27
1968_574           26
1960_340           24
2000_99-1884        2
2011_10-1195        1
Name: meta.case_id, Length: 6728, dtype: int64

In [100]:
utterances_df['meta.side'].isna().sum()

1121336

1 - petitioning party

0 - respondent

2 - amicus curiae

3 - unknown

In [102]:
utterances_df['meta.side'].value_counts()

1    416007
0    352773
3    260734
2     27459
Name: meta.side, dtype: int64

In [103]:
utterances_df['meta.side'].fillna(3, inplace=True)

In [104]:
utterances_df['meta.side'].value_counts()

3    1382070
1     416007
0     352773
2      27459
Name: meta.side, dtype: int64

## Obtaining embedded utterances 

We are now going to use gensim in order to obtain embedded utterances. We will use a word2vec model and then will pool the vectors for each utterance. 

To start, I am going to slice the dataframe down to a specific case and train on that. We will then use that to debug our model pipelines. 

In [35]:
case_1955_71 = utterances_df[utterances_df['meta.case_id'] == '1955_71']

In [36]:
case_1955_71.shape

(145, 17)

In [37]:
case_1955_71.head()

,timestamp,text,speaker,reply_to,conversation_id,meta.case_id,meta.start_times,meta.stop_times,meta.side,meta.timestamp,meta.cleaned,vectors,tokens,meta.win_side,meta.votes_side,speaker_type_A,speaker_type_J
0,None,"Number 71, Lonnie Affronti versus United State...",j__earl_warren,None,13127,1955_71,"[0.0, 7.624]","[7.624, 9.218]",None,0.0,number <number> lonnie affronti versus united ...,[],"[number, <number>, lonnie, affronti, versus, u...",0,"{'j__john_m_harlan2': 0, 'j__hugo_l_black': 0,...",0,1
1,None,May it please the Court.\nWe are here by writ ...,harry_f_murphy,13127__0_000,13127,1955_71,"[9.218, 11.538, 15.653, 22.722, 28.849, 33.575]","[11.538, 15.653, 22.722, 28.849, 33.575, 48.138]",1,9.218,may please court writ certiorari eighth circui...,[],"[may, please, court, writ, certiorari, eighth,...",0,"{'j__john_m_harlan2': 0, 'j__hugo_l_black': 0,...",1,0
2,None,Consecutive sentences.,j__william_o_douglas,13127__0_001,13127,1955_71,[48.138],[49.315],None,48.138,consecutive sentence,[],"[consecutive, sentence]",0,"{'j__john_m_harlan2': 0, 'j__hugo_l_black': 0,...",0,1
3,None,"Consecutive sentences.\nIn this case, the defe...",harry_f_murphy,13127__0_002,13127,1955_71,"[49.315, 51.844, 60.81, 67.083, 72.584, 89.839...","[51.844, 60.81, 67.083, 72.584, 89.839, 95.873...",1,49.315,consecutive sentence case defendant affronti i...,[],"[consecutive, sentence, case, defendant, affro...",0,"{'j__john_m_harlan2': 0, 'j__hugo_l_black': 0,...",1,0
4,None,Was the aggregate prison sentence was 20 or 25...,<INAUDIBLE>,13127__0_003,13127,1955_71,[174.058],[176.766],None,174.058,aggregate prison sentence <number> <number> year,[],"[aggregate, prison, sentence, <number>, <numbe...",0,"{'j__john_m_harlan2': 0, 'j__hugo_l_black': 0,...",0,0


In [44]:
from gensim.models.word2vec import Word2Vec

Separating the word2vec training into separate steps. We will use a maximum vocabulary size of 10k for our initial training. 

In [45]:
w2vmodel_1case = Word2Vec(max_vocab_size=10000, vector_size=300)

In [46]:
w2vmodel_1case.build_vocab(case_1955_71['tokens'], progress_per=10000)

In [49]:
case_1955_71['tokens'].shape

(145,)

In [51]:
w2vmodel_1case.train(case_1955_71['tokens'], total_examples=145, epochs=30)

(26141, 88380)

Repeating the same procedure to train the word 2 vec model for the entire dataset.

In [105]:
w2vmodel = Word2Vec(max_vocab_size=10000, vector_size=300)

In [106]:
w2vmodel.build_vocab(utterances_df['tokens'], progress_per=100000)

In [107]:
utterances_df.shape

(2178309, 17)

In [108]:
w2vmodel.train(utterances_df['tokens'], total_examples=2178309, epochs=30)

(1084870486, 1279023060)

## Using the Trained W2V model to create embeddings

We will combine the vectors for each word in an utterance by averaging them. 

In [109]:
# set your word to vec model here as a global variable to be used in the below function
w2v = w2vmodel

In [110]:
import numpy as np

def get_pooled_embedding(tokens):
    word_embeddings = [w2v.wv[tok] for tok in tokens if tok in w2v.wv]
    if len(word_embeddings) > 0:
        average = np.mean(word_embeddings, axis=0)
    else:
        average = np.zeros((300, ))
    return average

In [111]:
utterances_df['utterance_embedding'] = utterances_df['tokens'].apply(get_pooled_embedding)

In [85]:
case_1955_71['utterance_embedding'] = case_1955_71['tokens'].apply(get_pooled_embedding)

/var/folders/3l/_2dxvpjx083453g4qq1yr83w0000gn/T/ipykernel_62824/961153977.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  case_1955_71['sentence_embedding'] = case_1955_71['tokens'].apply(get_pooled_embedding)


In [112]:
utterances_df.head()

,timestamp,text,speaker,reply_to,conversation_id,meta.case_id,meta.start_times,meta.stop_times,meta.side,meta.timestamp,meta.cleaned,vectors,tokens,meta.win_side,meta.votes_side,speaker_type_A,speaker_type_J,utterance_embedding
0,None,"Number 71, Lonnie Affronti versus United State...",j__earl_warren,None,13127,1955_71,"[0.0, 7.624]","[7.624, 9.218]",3,0.0,number <number> lonnie affronti versus united ...,[],"[number, <number>, lonnie, affronti, versus, u...",0,"{'j__john_m_harlan2': 0, 'j__hugo_l_black': 0,...",0,1,"[-0.8707382, -1.7160745, -0.6207574, 0.2674471..."
1,None,May it please the Court.\nWe are here by writ ...,harry_f_murphy,13127__0_000,13127,1955_71,"[9.218, 11.538, 15.653, 22.722, 28.849, 33.575]","[11.538, 15.653, 22.722, 28.849, 33.575, 48.138]",1,9.218,may please court writ certiorari eighth circui...,[],"[may, please, court, writ, certiorari, eighth,...",0,"{'j__john_m_harlan2': 0, 'j__hugo_l_black': 0,...",1,0,"[0.51015204, -1.1017727, 0.3864366, -0.0528415..."
2,None,Consecutive sentences.,j__william_o_douglas,13127__0_001,13127,1955_71,[48.138],[49.315],3,48.138,consecutive sentence,[],"[consecutive, sentence]",0,"{'j__john_m_harlan2': 0, 'j__hugo_l_black': 0,...",0,1,"[1.5715005, -0.8253331, 2.721192, -0.25741312,..."
3,None,"Consecutive sentences.\nIn this case, the defe...",harry_f_murphy,13127__0_002,13127,1955_71,"[49.315, 51.844, 60.81, 67.083, 72.584, 89.839...","[51.844, 60.81, 67.083, 72.584, 89.839, 95.873...",1,49.315,consecutive sentence case defendant affronti i...,[],"[consecutive, sentence, case, defendant, affro...",0,"{'j__john_m_harlan2': 0, 'j__hugo_l_black': 0,...",1,0,"[0.0593424, -0.68631756, 0.1415781, -0.1229764..."
4,None,Was the aggregate prison sentence was 20 or 25...,<INAUDIBLE>,13127__0_003,13127,1955_71,[174.058],[176.766],3,174.058,aggregate prison sentence <number> <number> year,[],"[aggregate, prison, sentence, <number>, <numbe...",0,"{'j__john_m_harlan2': 0, 'j__hugo_l_black': 0,...",0,0,"[-0.26007193, -0.03844468, 0.79606074, -0.3663..."


In [113]:
utterances_df['utterance_embedding'][0].shape

(300,)

In [116]:
utterances_df = utterances_df[["utterance_embedding", "speaker_type_A", "speaker_type_J", "meta.side", "meta.win_side"]]

In [117]:
utterances_df.head()

,utterance_embedding,speaker_type_A,speaker_type_J,meta.side,meta.win_side
0,"[-0.8707382, -1.7160745, -0.6207574, 0.2674471...",0,1,3,0
1,"[0.51015204, -1.1017727, 0.3864366, -0.0528415...",1,0,1,0
2,"[1.5715005, -0.8253331, 2.721192, -0.25741312,...",0,1,3,0
3,"[0.0593424, -0.68631756, 0.1415781, -0.1229764...",1,0,1,0
4,"[-0.26007193, -0.03844468, 0.79606074, -0.3663...",0,0,3,0


In [119]:
utt_np_arr = utterances_df.to_numpy()

In [120]:
with open("utterances.npy", "wb") as f:
    np.save(f, utt_np_arr)

In [121]:
with open("utterances.npy", "rb") as f:
    test = np.load(f, allow_pickle=True)
print(test[0])

[array([-8.70738208e-01, -1.71607447e+00, -6.20757401e-01,  2.67447144e-01,
        -6.14075065e-02,  2.19559252e-01,  6.26445770e-01,  8.25892448e-01,
        -1.00384259e+00, -8.37345868e-02,  6.46695733e-01, -8.64544213e-01,
        -1.55116117e+00,  3.29568684e-01,  2.46760815e-01,  5.19752979e-01,
        -7.01112270e-01, -3.81856620e-01,  2.01811880e-01, -1.02813005e+00,
        -2.31853165e-02,  6.62830174e-01,  5.53926229e-01,  3.87815535e-02,
        -8.06790173e-01,  7.83012807e-01,  1.08295488e+00,  1.81906447e-01,
         1.32237315e+00,  2.98814178e-01, -3.00511897e-01, -6.92219734e-01,
         4.63237166e-01,  6.46214902e-01,  8.44015658e-01, -5.81416965e-01,
         1.56645846e+00,  5.40152669e-01, -4.65097725e-01,  7.96111345e-01,
        -7.22169876e-01, -7.86633432e-01, -7.08492279e-01, -1.52152598e-01,
        -8.63285124e-01,  9.08269942e-01,  6.47146761e-01,  6.08293861e-02,
        -3.55272889e-02, -1.20303655e+00,  2.68018395e-01, -1.22896940e-01,
         5.0

In [131]:
len(utt_np_arr)

2178309

In [132]:
utt_split = np.array_split(utt_np_arr, 6)

In [133]:
len(utt_split[0])

726103

In [134]:
utt_split[0].shape

(726103, 5)

In [136]:
counter = 0
for a in utt_split:
    counter += 1
    with open(f"utterances-{counter}.npy", "wb") as f:
        np.save(f, a)

In [135]:
for a in utt_split:
    print(a.shape)

(726103, 5)
(726103, 5)
(726103, 5)


## Grouping Utterances by Case for Case Level Classification

I wanted to try creating documents for each case in the event that we wanted to do case-level classification. 

Just going to concatenate all text for the cases. 

In [66]:
utt_df_cpy = utterances_df.copy(deep=True)

Also going to remove some columns to make this next part simpler...

In [72]:
utt_df_cpy.drop(columns=['speaker', 'reply_to', 'conversation_id',
                         'meta.start_times', 'meta.stop_times', 
                         'meta.speaker_type', 'meta.side', 
                         'meta.timestamp', 'vectors', 'timestamp', 
                         'meta.votes_side'])

,text,meta.case_id,meta.cleaned,meta.win_side
0,"Number 71, Lonnie Affronti versus United State...",1955_71,number <number> lonnie affronti versus united ...,0
1,May it please the Court.\nWe are here by writ ...,1955_71,may please court writ certiorari eighth circui...,0
2,Consecutive sentences.,1955_71,consecutive sentence,0
3,"Consecutive sentences.\nIn this case, the defe...",1955_71,consecutive sentence case defendant affronti i...,0
4,Was the aggregate prison sentence was 20 or 25...,1955_71,aggregate prison sentence <number> <number> year,0
...,...,...,...,...
2179509,-- has all sorts of meaning that you're not en...,2019_19-67,sort meaning youre endorsing youre saying aidi...,1
2179510,"No, Your Honor --",2019_19-67,honor,1
2179511,-- altogether?,2019_19-67,altogether,1
2179512,-- we are using the principles of complicity a...,2019_19-67,using principle complicity solicitation statut...,1


In [73]:
utt_df_cpy.groupby(['meta.case_id', 'meta.win_side'])['meta.cleaned'].apply(" ".join).reset_index()

,meta.case_id,meta.win_side,meta.cleaned
0,1955_10,0,number <number> commonwealth pennsylvania vers...
1,1955_102,0,minute remaining simply desire point brief dec...
2,1955_110,1,court dennis case one cited brief reading beca...
3,1955_111,1,number <number> gonzales versus hr landon dist...
4,1955_112,1,number <number> amos reece versus state georgi...
...,...,...,...
6711,2019_19-631,0,well hear argument next case <number> william ...
6712,2019_19-635,0,well hear argument next case <number> donald t...
6713,2019_19-67,1,well hear argument morning case <number> unite...
6714,2019_19-7,1,well hear argument first morning case <number>...
